# Block Model using `emg3d`

### Note regarding runtime

The following environment variables were set before starting Jupyter:
```
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1
```
This ensures that our code runs only on one thread. CPU-time is therefore the same as walltime (or even a tiny fraction smaller).

In [1]:
import emg3d
import numpy as np
import xarray as xr
from datetime import datetime
%load_ext memory_profiler

## Load model and survey

In [2]:
ds = xr.load_dataset('../block_model_and_survey.nc', engine='h5netcdf')

### Extract required info

In [3]:
# Get Mesh of Input model.
input_mesh = emg3d.TensorMesh(
    h=[ds.attrs['hx'], ds.attrs['hy'], ds.attrs['hz']],
    origin=ds.attrs['x0'],
)

# Create Input model.
input_model = emg3d.Model(
    grid=input_mesh,
    property_x=ds.attrs['resh_tg'],
    property_z=ds.attrs['resv_tg'],
    mapping='Resistivity',
)

# Get Survey
src = ds.attrs['src']
strength = ds.attrs['strength']
freq = ds.attrs['freq']
rec_x = ds.x.data[::2]
rec_y = ds.attrs['rec_y']
rec_z = ds.attrs['rec_z']

# Get source center (for meshing)
src_c = np.mean(src.reshape(-1, 2), 1)

## Computation mesh

In [4]:
mesh = emg3d.construct_mesh(
    frequency=freq,
    properties=[0.3, 1000, 100, 1000],
    center=[0, 0, 0],
    # domain=[[-10050, 10050], [-4000, 4000], [-3150, 0]],
    domain=[[-10450, 10450], [-5400, 5400], [-3150, 0]],
    min_width_limits=[100, 200, 50],
    max_buffer=50000,
    lambda_from_center=True,
)

# Refine x around source
mesh.h[0][122:124] = 50
mesh.h[0][124:-124] = 25
mesh.h[0][-124:-122] = 50
mesh.origin[0] += 400

# Refine y around source
mesh.h[1][32:-32] = 25
mesh.origin[1] += 1400

mesh = emg3d.TensorMesh(mesh.h, mesh.origin)
mesh

TensorMesh: 1,966,080 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x    256    -50,477.04     50,477.04     25.00  6,886.06    2.00
   y     80    -48,880.68     48,880.68     25.00 12,410.19    8.00
   z     96    -33,861.37     61,637.16     50.00 18,086.24    1.41

In [5]:
# Interpolate to computational mesh
model = input_model.interpolate_to_grid(mesh)

## `emg3d` computation

In [6]:
# Source field
sfield = emg3d.fields.get_source_field(mesh, src, freq, strength=strength)

In [7]:
options = {'sslsolver': False, 'return_info': True}
mem = %memit -o efield, info = emg3d.solve(model, sfield, verb=4, **options)
time = f"{info['time']:.0f} s"
ram = f"{(mem.mem_usage[0] - mem.baseline)/1024:.3f} GiB"
print(f"memory usage: {ram}")


:: emg3d START :: 10:02:21 :: v1.2.1

   MG-cycle       : 'F'                 sslsolver : False
   semicoarsening : True [1 2 3]        tol       : 1e-06
   linerelaxation : True [4 5 6]        maxit     : 50
   nu_{i,1,c,2}   : 0, 2, 1, 2          verb      : 4
   Original grid  : 256 x  80 x  96     => 1,966,080 cells
   Coarsest grid  :   2 x   5 x   3     => 30 cells
   Coarsest level :   7 ;   4 ;   5   

   [hh:mm:ss]  rel. error                  [abs. error, last/prev]   l s

       h_
      2h_ \                            /
      4h_  \                  /\      / 
      8h_   \          /\    /  \    /  
     16h_    \    /\  /  \  /    \  /   
     32h_     \/\/  \/    \/      \/    

   [10:02:58]   1.449e-02  after   1 F-cycles   [6.474e-03, 0.014]   4 1
   [10:03:30]   1.535e-03  after   2 F-cycles   [6.855e-04, 0.106]   5 2
   [10:03:59]   5.646e-04  after   3 F-cycles   [2.522e-04, 0.368]   6 3
   [10:04:31]   4.427e-05  after   4 F-cycles   [1.977e-05, 0.078]   4 1
   

In [8]:
# Extract (interpolate) Ex-field at receiver locations from the emg3d result.
egd = np.zeros((rec_x.size, 3), dtype=complex)
for i, y in enumerate(rec_y):
    egd[:, i] = efield.get_receiver((rec_x, y, rec_z, 0, 0))

## Save data

In [9]:
# Save the three lines
ds.line_1.data = np.vstack([egd[:, 0].real, egd[:, 0].imag]).ravel('F')
ds.line_2.data = np.vstack([egd[:, 1].real, egd[:, 1].imag]).ravel('F')
ds.line_3.data = np.vstack([egd[:, 2].real, egd[:, 2].imag]).ravel('F')

# Add info
ds.attrs['runtime'] = time
ds.attrs['n_procs'] = 1
ds.attrs['max_ram'] = ram
ds.attrs['n_cells'] = f"({mesh.shape_cells[0]} x {mesh.shape_cells[1]} x {mesh.shape_cells[2]}) - {mesh.n_cells}"
ds.attrs['n_nodes'] = 'N/A'
ds.attrs['n_dof'] = mesh.n_edges
ds.attrs['extent'] = (f"x = {mesh.nodes_x[0]:.1f}-{mesh.nodes_x[-1]:.1f}; " # Total mesh extent
                      f"y = {mesh.nodes_y[0]:.1f}-{mesh.nodes_y[-1]:.1f}; "
                      f"z = {mesh.nodes_z[0]:.1f}-{mesh.nodes_z[-1]:.1f}")
ds.attrs['min_vol'] = f"{np.min(mesh.cell_volumes):.1f}"
ds.attrs['max_vol'] = f"{np.max(mesh.cell_volumes):.1f}"
ds.attrs['machine'] = "Laptop; i7-6600U CPU@2.6 GHz x4; 15.5 GiB of memory, Ubuntu 20.04"
ds.attrs['version'] = f"emg3d v{emg3d.__version__}"
ds.attrs['date'] = datetime.today().isoformat()

# Save it under <{model}_{code}.nc>
ds.to_netcdf(f"../results/block_emg3d.nc", engine='h5netcdf')

In [10]:
emg3d.Report()

--------------------------------------------------------------------------------
  Date: Fri Sep 24 10:06:13 2021 CEST

                OS : Linux
            CPU(s) : 4
           Machine : x86_64
      Architecture : 64bit
               RAM : 15.5 GB
       Environment : Jupyter

  Python 3.8.0 | packaged by conda-forge | (default, Nov 22 2019, 19:11:38)
  [GCC 7.3.0]

             numpy : 1.20.3
             scipy : 1.7.1
             numba : 0.54.0
             emg3d : 1.2.1
           empymod : 2.1.2
            xarray : 0.19.0
        discretize : 0.7.0
              h5py : 3.3.0
        matplotlib : 3.4.3
              tqdm : 4.62.3
           IPython : 7.27.0

  Intel(R) Math Kernel Library Version 2020.0.4 Product Build 20200917 for
  Intel(R) 64 architecture applications
--------------------------------------------------------------------------------